In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


In [2]:
### Set up UDFs
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

,0
0,"UDF Yolo already exists, nothing added."


In [3]:
cursor.query(""" 
            CREATE UDF MonodepthDetection
            IMPL  '/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

CUDA is available.
 > 0: Tesla T4


/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


,0
0,UDF MonodepthDetection successfully added to t...
